In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

In [3]:
# Load Data
df = pd.read_csv('cover_data.csv')

In [4]:
# Seperate features and target
X = df.drop('class', axis=1)
y = df['class'] - 1 # convert to 0-6

In [5]:
# Identify columns
num_cols = [
    'Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
    'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points'
]

# The rest are binary
bin_cols = [col for col in X.columns if col not in num_cols]

In [7]:
# Scale only numerical columns
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

In [8]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [11]:
# Model creation
model = keras.Sequential([
    layers.Input(shape=(54,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(7, activation='softmax') # 7 output classes
])

In [12]:
# Model compile
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [17]:
# Model training
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data = (X_test, y_test),
    epochs=50,
    batch_size=256,
    callbacks=[early_stop],
    verbose=2
)

Epoch 1/50
1816/1816 - 7s - 4ms/step - accuracy: 0.7388 - loss: 0.6244 - val_accuracy: 0.7744 - val_loss: 0.5308
Epoch 2/50
1816/1816 - 11s - 6ms/step - accuracy: 0.7847 - loss: 0.5010 - val_accuracy: 0.7937 - val_loss: 0.4820
Epoch 3/50
1816/1816 - 10s - 5ms/step - accuracy: 0.8015 - loss: 0.4634 - val_accuracy: 0.8078 - val_loss: 0.4530
Epoch 4/50
1816/1816 - 10s - 5ms/step - accuracy: 0.8123 - loss: 0.4404 - val_accuracy: 0.8156 - val_loss: 0.4367
Epoch 5/50
1816/1816 - 6s - 3ms/step - accuracy: 0.8209 - loss: 0.4245 - val_accuracy: 0.8242 - val_loss: 0.4205
Epoch 6/50
1816/1816 - 5s - 3ms/step - accuracy: 0.8273 - loss: 0.4116 - val_accuracy: 0.8290 - val_loss: 0.4089
Epoch 7/50
1816/1816 - 6s - 3ms/step - accuracy: 0.8328 - loss: 0.4020 - val_accuracy: 0.8339 - val_loss: 0.4012
Epoch 8/50
1816/1816 - 9s - 5ms/step - accuracy: 0.8366 - loss: 0.3938 - val_accuracy: 0.8334 - val_loss: 0.4002
Epoch 9/50
1816/1816 - 6s - 3ms/step - accuracy: 0.8394 - loss: 0.3872 - val_accuracy: 0.8404

In [18]:
# Model evaluation
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {test_acc:.4f}")

3632/3632 - 6s - 2ms/step - accuracy: 0.8726 - loss: 0.3153
Test Accuracy: 0.8726


In [19]:
# More detailed analysis

y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)

print(classification_report(y_test, y_pred_classes, target_names=[
    "Spruce/Fir", "Lodgepole Pine", "Ponderosa Pine", "Cottonwood/Willow", "Aspen", "Douglas-fir", "Krummholz"
]))

3632/3632 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step
                   precision    recall  f1-score   support

       Spruce/Fir       0.87      0.87      0.87     42368
   Lodgepole Pine       0.89      0.90      0.89     56661
   Ponderosa Pine       0.83      0.90      0.87      7151
Cottonwood/Willow       0.83      0.74      0.78       549
            Aspen       0.75      0.55      0.63      1899
      Douglas-fir       0.77      0.70      0.73      3473
        Krummholz       0.87      0.90      0.88      4102

         accuracy                           0.87    116203
        macro avg       0.83      0.79      0.81    116203
     weighted avg       0.87      0.87      0.87    116203



In [23]:
# Model save
model.save('forest_cover_model.keras')